In [7]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/cambrian/'
data_folder = parent_folder + 'data/'
from hydra import compose, initialize
with initialize(config_path="../../conf"):
    # Compose the configuration
    cfg = compose(config_name="TopicClusterMapping.yaml")

import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_6305/4032682437.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../../conf"):


In [3]:
from pymongo import MongoClient

In [4]:
import pandas as pd

In [15]:
os.environ.get("MONGO_DB_NAME")

'insight_db'

In [21]:
import os
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder

class MongoDBConnection:
    _instance = None
    _client = None
    # _server = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(MongoDBConnection, cls).__new__(cls)
            
            ssh_host = os.environ.get("SSH_HOST")
            ssh_port = int(os.environ.get("SSH_PORT", 22))
            ssh_username = os.environ.get("SSH_USERNAME")
            ssh_password = os.environ.get("SSH_PASSWORD")
            ssh_pkey = '/Users/ravi.tej/Downloads/insight-user-beta-env-keypair.pem'  # path to private key
            
            mongo_host = os.environ.get("MONGO_HOST", "localhost")
            # mongo_host = 'localhost'
            mongo_port = int(os.environ.get("MONGO_PORT", 27017))
            # mongo_port = 27017
            mongo_username = os.environ.get("MONGO_USERNAME")
            mongo_password = os.environ.get("MONGO_PASSWORD")
            mongo_db_name = os.environ.get("MONGO_DB_NAME")

            cls._server = SSHTunnelForwarder(
                (ssh_host, ssh_port),
                ssh_username=ssh_username,
                # ssh_password=ssh_password,
                ssh_pkey=ssh_pkey,
                remote_bind_address=(mongo_host, mongo_port)
            )
            cls._server.start()

            cls._instance._client = MongoClient(
                host=mongo_host,
                # port=cls._server.local_bind_port,  # Note: Use the local_bind_port
                port = mongo_port,
                username=mongo_username,
                password=mongo_password
            )
        return cls._instance

    @classmethod
    def get_database(cls, db_name=None):
        db_name = db_name or os.environ.get("MONGO_DB_NAME")
        # db_name = 'insight_db'
        return cls._instance._client[db_name]

    @classmethod
    def close(cls):
        if cls._server is not None:
            cls._server.stop()

In [12]:
# class MongoDatabaseConnection:
#     _instance = None
#     _client = None

#     def __new__(cls):
#         if cls._instance is None:
#             cls._instance = super(MongoDatabaseConnection, cls).__new__(cls)
#             cls._instance._client = MongoClient(
#                 # host=os.environ.get("MONGO_HOST"),
#                 host='localhost',
#                 # port=int(os.environ.get("MONGO_PORT", 27017)),
#                 port=27017,
#                 # username=os.environ.get("MONGO_USER"),
#                 # password=os.environ.get("MONGO_PASSWORD")
#             )
#         return cls._instance

#     @classmethod
#     def get_database(cls, db_name=None):
#         db_name = db_name or os.environ.get("MONGO_DB_NAME")
#         return cls._instance._client[db_name]

In [22]:
db = MongoDBConnection().get_database()

In [23]:
print(db)

Database(MongoClient(host=['insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:27017'], document_class=dict, tz_aware=False, connect=True), 'insight_db')


In [24]:
articles_collection = db['articles']

In [34]:
articles = articles_collection.find({})

In [25]:
articles = articles_collection.find(
    {"is_premium_article": False},
    { "published_time": 1, "source_id": 1, "_id": 1}
)

In [26]:
df = pd.DataFrame(list(articles))

ServerSelectionTimeoutError: insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:27017: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 65252f4653b54926c056f5f3, topology_type: Unknown, servers: [<ServerDescription ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:27017: timed out')>]>

In [ ]:
df

In [52]:
df['_id'] = df['_id'].apply(lambda x: str(x))

In [ ]:
from sql

In [53]:
df.iloc[0]._id

'65236ceccd871c6ac1cde043'

In [5]:
def get_articles_metadata(article_ids):
    db = MongoDBConnection().get_database()
    articles_collection = db['articles']  # Assuming your collection is named 'articles'
    article_ids_objectids = [ObjectId(id1) for id1 in article_ids]
    print(article_ids_objectids)
    # Query to fetch the required data
    articles = articles_collection.find(
        {"_id": {"$in": article_ids_objectids}},
        {"published_time": 1, "source_id": 1, "_id": 1}
    )

    return list(articles)

In [12]:
from bson.objectid import ObjectId

In [57]:


# # Convert article_ids to ObjectIds


articles = articles_collection.find(
    {"_id": {"$in": [ObjectId('65236ceccd871c6ac1cde043')]}},
    { "published_time": 1, "source_id": 1, "_id": 1}
)

DEBUG:paramiko.transport:Sending global request "keepalive@lag.net"


In [9]:
print(articles_collection)

Collection(Database(MongoClient(host=['insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:52863'], document_class=dict, tz_aware=False, connect=True), 'insight_db'), 'articles')


In [8]:
import logging
logging.basicConfig(level=logging.DEBUG)

DEBUG:paramiko.transport:Sending global request "keepalive@lag.net"


In [13]:
article_ids = ['65236ceccd871c6ac1cde043']

In [14]:
get_articles_metadata(article_ids)

[ObjectId('65236ceccd871c6ac1cde043')]


ServerSelectionTimeoutError: insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:52863: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 6523d8293c434a560af8d79b, topology_type: Unknown, servers: [<ServerDescription ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 52863) server_type: Unknown, rtt: None, error=NetworkTimeout('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:52863: timed out')>]>

In [28]:
non_premium_articles = articles_collection.find(
    {"is_premium_article": False},
    {"article_id": 1, "_id": 0}
)

In [31]:
article_ids = [article['article_id'] for article in non_premium_articles]

ServerSelectionTimeoutError: insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:49201: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 651ea579f270cc2744384796, topology_type: Unknown, servers: [<ServerDescription ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 49201) server_type: Unknown, rtt: None, error=NetworkTimeout('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:49201: timed out')>]>